In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from importlib import reload

In [3]:
import drivers, lib
from time import sleep
import pickle

In [57]:
from lib import data_management as dm

In [5]:
# reload(drivers.Agilent_EXA)
from drivers.Agilent_EXA import *
# reload(drivers.KeysightAWG)
from drivers.KeysightAWG import KeysightAWG, WaveformType
from drivers.E8257D import E8257D

In [6]:
exa = Agilent_EXA_N9010A("SA", "EXA")
awg = KeysightAWG("AWG2")
mwsrc = E8257D("MXG")

In [40]:
reload(lib.iq_mixer_calibration)
from lib.iq_mixer_calibration import *

In [41]:
cal = IQCalibrator(awg, exa, mwsrc, "TEST", -20)
res = cal.calibrate(lo_frequency=5e9, if_frequency=100e6, lo_power=10,
                    ssb_power=-20, iterations=5, minimize_iterlimit=20,sa_res_bandwidth=100,)
#                     initial_guess=res.get_optimization_results()[0])

Phase:  [-0.52501062] [-20.212928771972656, -93.98870086669922, -111.54064178466797], 

In [36]:
print(res)

Calibration data for mixer TEST
Parameters: {'ssb_power': -20, 'lo_frequency': 5000000000.0, 'lo_power': 10, 'if_frequency': 100000000.0}
Optimization results: {'dc': -97.870506286621094, 'if': [-19.039522171020508, -81.36407470703125, -103.27525329589844]}
Optimization time: 0 h 4 m 3.0 s


In [42]:
exa.setup_swept_sa(4.9e9, 800e6, nop=501, rbw=10e3)
exa.set_continuous()

In [44]:
mwsrc.set_power(10)

In [48]:
ndarray(0)

array([], dtype=float64)

In [49]:
concatenate((_, [1,4,5]))

array([ 1.,  4.,  5.])

In [61]:
reload(dm)

<module 'lib.data_management' from 'C:\\Users\\labiks\\Documents\\Gleb scripts\\lib\\data_management.py'>

In [62]:
dm.save_IQMX_calibration(res)

In [64]:
dm.load_IQMX_calibration_database("TEST", -20)

{frozenset({('if_frequency', 100000000.0),
            ('lo_frequency', 5000000000.0),
            ('lo_power', 10),
            ('ssb_power',
             -20)}): <lib.iq_mixer_calibration.IQCalibrationData at 0x817e6d8>}